In [13]:
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep
import time
import random
import os
from urllib.request import urlretrieve



requests.DEFAULT_RETRIES = 5  # 增加重连次数
s = requests.session()
s.keep_alive = False  # requsets库里面http connection 是 keep alive 的，一定要把它关了，否则就卡死了
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/'
                  '537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36'
}
html = requests.get('https://www.videvo.net/stock-video-footage/video/sort/random/freeclips/yes/',
                    headers=header).text
bs = BeautifulSoup(html, 'html.parser')
video_ids_naive = bs.find_all(['div'], {'class': 'video-thumb-inner'})
reg = r'<a href="(.*?)">'
video_ids = re.findall(reg, str(video_ids_naive))  # 这里以及成功找到每一个小图片的相对地址

video_urls = []
start_url = 'https://www.videvo.net'
for video_id in video_ids:
    video_urls.append(start_url + video_id)  # 这里得到封面上每个视频的绝对地址的列表，我们还要再进一步点进去

# ---------------------------下面这一部要花不少时间，因为要打开每一个页面，然后下载里面的视频----------------------- #
downloadable_urls = []
name_list = []
for video_url in tqdm(video_urls):  # 第一个tqdm用于把可下载的视频的网址和名字保存下来，全部扒下来以后先sleep(3)让requests休息一下
    sleep(0.5 + random.random())  # 适当延时一会，规避网站反爬虫机制
    html = requests.get(video_url, headers=header).text  # 图片点进去的页面，我们要在里面找到视屏播放的地址， 结尾一般是mp4结构
    reg = r'<source type="video/mp4" src="(.*?)">'
    downloadable_urls.append(re.findall(reg, html)[0])  # 每个图片的下载路径确实是找到了，但是有一些还是相对路径，非常令人恼火

    reg = '<span itemprop="name">(.*?)</span>'
    name_list.append(re.findall(reg, html)[0])  # 这里把每个视频的名字也爬下来

downloadable_urls_robust = []
base_url = 'https://cdn.videvo.net'
for url in downloadable_urls:
    if not str(url).startswith('https://'):
        url = base_url + url
    downloadable_urls_robust.append(url)




  0%|                                                                                           | 0/75 [00:00<?, ?it/s]


  1%|█                                                                                  | 1/75 [00:03<03:56,  3.20s/it]


  3%|██▏                                                                                | 2/75 [00:06<03:45,  3.09s/it]


  4%|███▎                                                                               | 3/75 [00:08<03:35,  2.99s/it]


  5%|████▍                                                                              | 4/75 [00:35<12:03, 10.19s/it]


  7%|█████▌                                                                             | 5/75 [00:39<09:33,  8.19s/it]


  8%|██████▋                                                                            | 6/75 [00:42<07:47,  6.77s/it]


  9%|███████▋                                                                           | 7/75 [00:46<06:34,  5.81s/it]


 11%|████████▊       

In [14]:
for name in name_list:
    print(name)

Zouave Statue Partially Submerged in Floodwater
Doctor Uses Stethoscope on Patient
CU Hospital Patient and Visitor Hold Hands
Melting Glacier Ice Close Up
Cyclist wearing face mask
Nigeria Slum 09
Pull Focus on Waterfall
Beer Stein Bubbles Panning Shot
Digital Mixer Motorized Faders (Close)
Busy Road in Downtown Seoul
Construction Site Timelapse Bangkok
Portsmouth Harbour at Dusk
Lava Lamp Close-up
Panning Over Da Nang Vietnam
Bridge and Creek 
Panning Close Up Clicking Youtube Upload Video Button
Space Rocket Launch 5
Stitching on Baseball Glove
Distant Waterfall on Mt Rainier
Purple Sunset 
Gaul Helmet Animation
Indian Men Waving Ceremonial Candelabras
Yellow and Green Leaves in Sunlight
Car Lights Out of Focus in New York
Lighthouse
Lady Performing Tai Chi
Street Artists at Venice Beach
Brandenburg Gate Berlin
Rising Hyperlapse Over Danang Vietnam
Burning Logs Slow Motion 4K 09
Illuminated Temple of Athena Nike
Clinking Red Wine Glasses Together
Chess Pieces 10
Record Spinning Rack 

In [15]:
for url in downloadable_urls_robust:
    print(url)

https://cdn.videvo.net/videvo_files/converted/2018_02/videos/180131_A_22.mp480848_jw.mp4
https://cdn.videvo.net/videvo_files/video/free/2020-02/small_watermarked/200129_02_Medical_Ward_4k_094_preview.mp4
https://cdn.videvo.net/videvo_files/video/free/2020-02/small_watermarked/200129_02_Medical_Ward_4k_053_preview.mp4
https://cdn.videvo.net/videvo_files/video/free/2019-04/small_watermarked/190408_01_Alaska_Landscapes1_13_preview.mp4
https://cdn.videvo.net/videvo_files/video/free/2020-03/small_watermarked/200323 - Coronavirus_01_4k_010_preview.mp4
https://cdn.videvo.net/videvo_files/video/free/2019-09/small_watermarked/190802_17a_HD_09_preview.mp4
https://cdn.videvo.net/videvo_files/converted/2017_04/videos/170216A_088_MilfordSoundsRain_1080p.mp449155.mp4
https://cdn.videvo.net/videvo_files/converted/2014_08/videos/Close_up_stein_pan.mov72171.mp4
https://cdn.videvo.net/videvo_files/converted/2014_12/videos/Digital_Mixer__Motor_Faders_CLOSE.mp424876.mp4
https://cdn.videvo.net/videvo_files

In [16]:
path = 'video'
if path not in os.listdir():
    os.mkdir(path)  # 这里是视频的下载地址

#  正式进入下载阶段
for i in tqdm(range(min(len(name_list), len(downloadable_urls_robust)))):
    print("正在下载第 %d 个视频，视频名: %s" % (i + 1, name_list[i]))
    sleep(0.5 + random.random())  # 休息一下，反爬
    movie_url = downloadable_urls_robust[i]
    movie_name = name_list[i]
    downsize = 0
    start_time = time.time()
    req = requests.get(movie_url, headers=header, stream=True, verify=False)
    file_path = path + '/%s.mp4' % movie_name
    with(open(file_path, 'wb')) as f:
        for chunk in req.iter_content(chunk_size=10000):
            if chunk:
                f.write(chunk)
                downsize += len(chunk)
                line = 'downloading %d KB/s - %.2f MB， 共 %.2f MB'
                line = line % (
                    downsize / 1024 / (time.time() - start_time), downsize / 1024 / 1024, downsize / 1024 / 1024)
                print(line)





  0%|                                                                                           | 0/75 [00:00<?, ?it/s]

正在下载第 1 个视频，视频名: Zouave Statue Partially Submerged in Floodwater


D:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


downloading 14 KB/s - 0.01 MB， 共 0.01 MB
downloading 26 KB/s - 0.02 MB， 共 0.02 MB
downloading 39 KB/s - 0.03 MB， 共 0.03 MB
downloading 52 KB/s - 0.04 MB， 共 0.04 MB
downloading 62 KB/s - 0.05 MB， 共 0.05 MB
downloading 74 KB/s - 0.06 MB， 共 0.06 MB
downloading 86 KB/s - 0.07 MB， 共 0.07 MB
downloading 96 KB/s - 0.08 MB， 共 0.08 MB
downloading 107 KB/s - 0.09 MB， 共 0.09 MB
downloading 119 KB/s - 0.10 MB， 共 0.10 MB
downloading 130 KB/s - 0.10 MB， 共 0.10 MB
downloading 142 KB/s - 0.11 MB， 共 0.11 MB
downloading 152 KB/s - 0.12 MB， 共 0.12 MB
downloading 163 KB/s - 0.13 MB， 共 0.13 MB
downloading 174 KB/s - 0.14 MB， 共 0.14 MB
downloading 185 KB/s - 0.15 MB， 共 0.15 MB
downloading 195 KB/s - 0.16 MB， 共 0.16 MB
downloading 205 KB/s - 0.17 MB， 共 0.17 MB
downloading 216 KB/s - 0.18 MB， 共 0.18 MB
downloading 218 KB/s - 0.19 MB， 共 0.19 MB
downloading 228 KB/s - 0.20 MB， 共 0.20 MB
downloading 238 KB/s - 0.21 MB， 共 0.21 MB
downloading 248 KB/s - 0.22 MB， 共 0.22 MB
downloading 257 KB/s - 0.23 MB， 共 0.23 MB





  1%|█                                                                                  | 1/75 [00:02<02:29,  2.02s/it]

...




 99%|████████████████████████████████████████████████████████████████████████████████▉ | 74/75 [05:59<00:02,  2.35s/it]

正在下载第 75 个视频，视频名: Couple Walking along Westminster Bridge


D:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


downloading 15 KB/s - 0.01 MB， 共 0.01 MB
downloading 29 KB/s - 0.02 MB， 共 0.02 MB
downloading 43 KB/s - 0.03 MB， 共 0.03 MB
downloading 56 KB/s - 0.04 MB， 共 0.04 MB
downloading 69 KB/s - 0.05 MB， 共 0.05 MB
downloading 81 KB/s - 0.06 MB， 共 0.06 MB
downloading 93 KB/s - 0.07 MB， 共 0.07 MB
downloading 105 KB/s - 0.08 MB， 共 0.08 MB
downloading 118 KB/s - 0.09 MB， 共 0.09 MB
downloading 129 KB/s - 0.10 MB， 共 0.10 MB
downloading 141 KB/s - 0.10 MB， 共 0.10 MB
downloading 153 KB/s - 0.11 MB， 共 0.11 MB
downloading 165 KB/s - 0.12 MB， 共 0.12 MB
downloading 176 KB/s - 0.13 MB， 共 0.13 MB
downloading 188 KB/s - 0.14 MB， 共 0.14 MB
downloading 200 KB/s - 0.15 MB， 共 0.15 MB
downloading 210 KB/s - 0.16 MB， 共 0.16 MB
downloading 222 KB/s - 0.17 MB， 共 0.17 MB
downloading 233 KB/s - 0.18 MB， 共 0.18 MB
downloading 244 KB/s - 0.19 MB， 共 0.19 MB
downloading 255 KB/s - 0.20 MB， 共 0.20 MB
downloading 266 KB/s - 0.21 MB， 共 0.21 MB
downloading 276 KB/s - 0.22 MB， 共 0.22 MB
downloading 287 KB/s - 0.23 MB， 共 0.23 MB




100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [06:01<00:00,  4.82s/it]
